# STI/NTRS/PubSpace Harvester

This notebook sends a query to STI/NTRS API for harvesting bibliographic records. Data returned are processed through the ADS Reference Service to weed out records already existing in ADS. The Excel workbook output can be reviewed and curated for ingest to ADS.
___
#### STI Harvest
1. Set name for output file (date + category) and filepath
2. Select API endpoint for querying (regular STI Collection vs Pubspace) and parameters
3. Connect to STI API and pull/store data locally

#### ADS Bibcode Matching
4. Tranform STI results into ref strings
5. Query the ADS Reference Service API with ref strings, return bibcode matches
6. Output the records with bibcode results
___
NOTEBOOK OUTPUT: 
- Excel workbook; "{name}_data.xlsx" includes:
   - bibliographic metadata retrieved from STI, curated for ADS
   - index label: STI or CHORUS
   - reference resolver data: refstring, score, and bibcode if matched

In [ ]:
import requests
import json
import pandas as pd
import numpy as np
import math
import unicodedata
from pyingest.serializers.classic import Tagged
import re
import csv
import os
import datetime

# -- Set the year/name of output file
name = "2023"
# name = "no_pub"  # search for STI records that don't have a pubdate

# -- Set local filepath
filepath = "/Users/sao/Documents/Python-Projects/NASA_STI/data/"
filename = name + "_data.xlsx"

In [ ]:
# Combine the filepath and file name to get the full path to the Excel file
infile = os.path.join(filepath, filename)

# Check if the file exists
if not os.path.exists(infile):
    # Create an empty DataFrame and save it as the Excel file
    column_names = ["ident", "authors", "affiliations", "title", "pubdate", "publication", "abstract", "properties", "collection", "keywords", "subjectcategory", "comment", "source"]
    data = pd.DataFrame(columns=column_names)
    data.to_excel(infile, index=False)

# Input query parameters
gte=name+"-01-01"  # Greater than or equal to the first of the year
lte=name+"-12-31"  # Less than or equal to the last of the year

if name == "no_pub":
    today = datetime.date.today()
    year = today.year
    month = today.month

    # Generate the start and end dates for the current month
    start_date = f"{year}-{month:02d}-01"  #YYYY-MM-01
    end_date = f"{year}-{month:02d}-{calendar.monthrange(year, month)[1]}"  #YYYY-MM-31

    # Set the parameters - search modified date of the current month
    params = {
        "modified": {
            "gte": start_date,
            "lte": end_date
        },
        "page": {"size": 100}
    }

elif name == "2013": # search for records with pubdate less than 2013-12-31
    params = {
         "published": {
             "lte":lte
         },
        "page": {"size":100}
    }
else: # search for records with pubdate within year given
    params = {
         "published": {
             "gte":gte,
             "lte":lte
         },
        "page": {"size":100}
    }


## NTRS API Query
Retrieves all records from above parameters. 
Then grabs a handpicked set of desired metadata fields.

In [ ]:
# API Setup
base_url = "https://ntrs.nasa.gov/api"
path = "/pubspace/search"
api_url = base_url + path
MAX_RECORDS = 100

def get_batch(api_url, params):
    get_header = {'Accept': 'text/plain',
                  'Content-type': 'application/json'}
    buff = requests.post(api_url, headers=get_header, data=json.dumps(params)).json()
    return buff

def get_records(url, params):
    records = []
    
    # Do the first query
    try:
        batch = get_batch(url, params)
    except Exception as err:
        raise Exception("Request to STI blew up: %s" % err)
        
    # Count of total records
    totrecs = batch['stats']['total']
 
    # Store the first batch of records  
    records += batch['results']
    
    # Print count of total records and pages
    num_paginates = int(math.ceil((totrecs) / (1.0*MAX_RECORDS)))
    print("> Total records: %d \n > Total pages: %d" % (totrecs, num_paginates))
          
    # Continue requests
    offset = MAX_RECORDS
    for i in range(num_paginates):
        params['page']['from'] = offset
        try:
            batch = get_batch(url, params)
        except Exception as err:
            raise URLError("Request to STI blew up: %s" % err)
        records += batch['results']
        offset += MAX_RECORDS
    return records

In [ ]:
# Run API Request
from_sti = get_records(api_url, params)

# Normalize json results to generate DataFrame
dt = pd.json_normalize(from_sti, meta=['title'])

# List of desired fields
desired_fields = [
        "id",
        "subjectCategories",
        "fundingNumbers",
        "authorAffiliations",
        "title",
        "stiType",
        "abstract",
        "publications",
        "center.code",
        "keywords",
        "sourceIdentifiers",
        "meetings",
        "downloads",
        "index"
    ]

# Grab desired fields if present in dt
df = dt[[col for col in desired_fields if col in dt.columns]]

# Drop rows where Document ID is null
df = df.dropna(subset=['id'])

# Drop duplicates by Document ID
df = df.drop_duplicates(subset=['id'], keep='last')

print("Harvested %d records"%(len(df)))

## Metadata Wrangling
Transforms exisiting STI metadata into ADS desired metadata:
Title, Abstract, Keywords, Collection, Subject Category (Earth Science), Authors and their affiliations, STI id, Funding numbers, NASA Center of origin, NTRS index source (STI v. CHORUS), publication info, links/dois.

In [ ]:
# TITLE/T
lsT = [t if t else '' for t in df['title'] if 'abstract' in df.columns]

# ABSTRACT/B
lsB = [b if b else '' for b in df['abstract'] if 'abstract' in df.columns]

# KEYWORDS/K - Put STI Subject Category into %K
lsK = []
if 'keywords' in df.columns and 'subjectCategories' in df.columns:
    for s, k, in zip(df['subjectCategories'], df['keywords']):
        keywords = ''
        if isinstance(s, list):
            keywords += ', '.join(s)
        if isinstance(k, list):
            keywords += ', ' + ', '.join(k)
        lsK.append(keywords.lstrip(', '))
else:
    lsK = [''] * len(df)

# COLLECTION/W & SUBJECT CATEGORY/Q - Assign ADS %W and %Q based on STI Subject Categories
AST_cats = ["Astronomy", "Astrophysics"]
PHY_cats = ["Physics (General)", "Acoustics", "Atomic And Molecular Physics",
    "Nuclear Physics", "Optics", "Plasma Physics", "Solid-State Physics",
    "Physics Of Elementary Particles And Fields", "Astronautics (General)",
    "Astrodynamics", "Ground Support Systems And Facilities (Space)",
    "Launch Vehicles And Launch Operations", "Space Transportation And Safety",
    "Space Communications, Spacecraft Communications, Command And Tracking",
    "Spacecraft Design, Testing And Performance", "Spacecraft Instrumentation And Astrionics",
    "Spacecraft Propulsion And Power", "Engineering (General)",
    "Communications And Radar", "Electronics And Electrical Engineering",
    "Fluid Mechanics And Thermodynamics", "Instrumentation And Photography",
    "Lasers And Masers", "Mechanical Engineering",
    "Quality Assurance And Reliability", "Structural Mechanics"]
ES_cats = ["Geosciences (General)", "Earth Resources And Remote Sensing",
    "Energy Production And Conversion", "Environment Pollution",
    "Geophysics", "Meteorology And Climatology", "Oceanography"]
# PS_cats = ["Space Sciences (General)", "Lunar and Planetary Science and Exploration", "Exobiology"]
# BPS_cats = ["Life Sciences (General)", "Aerospace Medicine", "Behavioral Sciences", "Man/System Technology and Life Support"]
# HP_cats = ["Solar Physics", "Space Radiation"]

lsW = []  # Collection
lsQ = []  # Subject category (Earth Sciencce)
if 'subjectCategories' in df.columns:
    for subjects in df['subjectCategories']:
        if isinstance(subjects, float) and np.isnan(subjects):
            # Skip iteration if the value is NaN
            lsW.append('GEN')
            lsQ.append('')
            continue
        if any(subject in AST_cats for subject in subjects):
            lsW.append('AST')
            lsQ.append('')
        elif any(subject in ES_cats for subject in subjects):
            lsW.append('GEN')
            lsQ.append('Earth Science')
        elif any(subject in PHY_cats for subject in subjects):
            lsW.append('PHY')
            lsQ.append('')
        else:
            lsW.append('GEN')
            lsQ.append('')

In [ ]:
# AUTHORS/A & AFFILIATIONS/F

# Function to reformat author names if not already in format "Last, First"
    # Also appends periods '.' to initials if necessary
def reformat_author(author_name):
   
    # If comma in author name, assume it's in "Last, First" format already; no need to reformat
    if ', ' in author_name:
        name_parts = author_name.split(', ')
        last_name = name_parts[0]
        
        if len(name_parts) > 1:
            first_middle_parts = name_parts[1].split()
            first_name = first_middle_parts[0] if first_middle_parts else ''
            middle_name = ' '.join(first_middle_parts[1:]) if len(first_middle_parts) > 1 else ''
        else:
            first_name = ''
            middle_name = ''

        # Check and add period to first name if necessary
        if len(first_name) == 1 and not first_name.endswith('.'):
            first_name += '.'

        # Check and add period to middle name if necessary
        initials = middle_name.split()
        for i in range(len(initials)):
            if len(initials[i]) == 1 and not initials[i].endswith('.'):
                initials[i] += '.'
        middle_name = ' '.join(initials)

        formatted_name = "{}, {}".format(last_name, first_name)
        if middle_name:
            formatted_name += " {}".format(middle_name)

        return formatted_name
    
    # Reformat if there's no comma in author_name
    else:
        name_list = author_name.split()
        if len(name_list) > 0:
            first_name = name_list[0]
            last_name = ""
            middle_name = ""
            suffix = ""

            # Check for suffixes
            suffixes = ["Jr.", "Sr.", "II", "III"]
            if len(name_list) > 1:
                last_word = name_list[-1]
                if last_word in suffixes:
                    suffix = last_word
                    name_list = name_list[:-1]

                # Format name parts
                if len(name_list) > 1:
                    last_name = name_list[-1]
                    middle_name = " ".join(name_list[1:-1])
                else:
                    last_name = name_list[0]

                # Check and add period to first name if necessary
                if len(first_name) == 1 and not first_name.endswith('.'):
                    first_name += '.'

                # Check and add period to middle name if necessary
                if len(middle_name) == 1 and not middle_name.endswith('.'):
                    middle_name += '.'

            # Construct formatted name
            formatted_name = "{}, {}".format(last_name, first_name)
            if middle_name:
                formatted_name += " {}".format(middle_name)
            if suffix:
                formatted_name += ", {}".format(suffix)
            return formatted_name


# Grab Author/Affiliation Metadata
lsA = []  # Authors List
lsF = []  # Affiliations List

if 'authorAffiliations' in df.columns:
    for authors in df['authorAffiliations']:
        row_authors = []
        row_affils = []

        if isinstance(authors, list):
            for entry in authors:
                author = entry['meta']['author'].get('name', '')  # Concat author names
                reformatted = reformat_author(author)
                if reformatted:  # Only append non-empty author names
                    row_authors.append(reformatted)

                organization = entry['meta'].get('organization', {})  # Concat aff names
                aff_name = organization.get('name', '')
                aff_loc = organization.get('location', '')  # Concat aff locations
                affil = ', '.join(filter(None, [aff_name, aff_loc]))

                orcid = entry['meta']['author'].get('orcidId', '')  # Concat orcids
                if orcid:
                    affil += ' <ID system="ORCID">{}</ID>'.format(orcid)

                if affil:  # Only append non-empty affiliations
                    row_affils.append(affil)
                    
            lsA.append(row_authors)
            lsF.append(row_affils)

        else:
            lsA.append('')
            lsF.append('')

In [ ]:
# STI Stuff
sources = ["CHORUS" if "chorus" in i else "STI" for i in df['index']]
STI_ids = [d if d else '' for d in df['id']]
lsX = [f"NASA Center: {c}" if pd.notna(c) else '' for c in df['center.code'] if 'center.code' in df.columns]
    
# Funding Numbers
fundingNums = []
if 'fundingNumbers' in df.columns:
    for numbers in df['fundingNumbers']:
        if isinstance(numbers, list):
            for entry in numbers:
                if isinstance(entry, dict) and 'type' in entry and 'number' in entry:
                    fundType = entry['type']
                    fundNum = entry['number']
                    fundingNums.append('{}: {}'.format(fundType, fundNum))
                else:
                    fundingNums.append('')

In [ ]:
# PUBDATE/D, PUBLICATION/J, LINK PROPERTIES/I 

# Initialize metadata lists
pubs_ls = [] # Pubnames
vols_ls = [] # Volumes
dois_ls = [] # DOIs
lsD = []  # Pubdates
lsJ = []  # Journal/Pub
lsI = []  # Properties/Links

for pubs, docID, idents, dwnlds, funds  in zip(df['publications'], STI_ids, df['sourceIdentifiers'], df['downloads'], df['fundingNumbers']):
    j = ''
    links = ''
    pubnames = ''
    volumes = ''
    dois = ''
    
    for entry in pubs:
        pubdate = entry.get('publicationDate', '')[:10]
        publication_name = entry.get('publicationName', '')
        vol = entry.get('volume', '')
        issue = entry.get('issue', '')
        publisher = entry.get('publisher', '')
        issn = entry.get('issn', '')
        eissn = entry.get('eissn', '')
        isbn = entry.get('isbn', '')
        eisbn = entry.get('eisbn', '')
        url = entry.get('url', '')
        doi = entry.get('doi', '')
            
        if publication_name:
            j += publication_name.rstrip(" ")
            pubnames += publication_name.rstrip(" ")
        if vol:
            j += ', Vol. {}'.format(vol)
            volumes += vol
        if issue:
            j += ', Issue {}'.format(issue)
        if publisher:
            j += ', Published by {}'.format(publisher.lstrip(" "))
        if pubdate:
            j += ', {}'.format(pubdate[:4].rstrip(" "))
        if issn:
            j += ', ISSN: {}'.format(issn)
        if eissn:
            j += ', eISSN: {}'.format(eissn)
        if isbn:
            j += ', ISBN: {}'.format(isbn)
        if eisbn:
            j += ', eISBN: {}'.format(isbn)
        j = j.lstrip(', ')

        # Concat link info from publications field
        doi = doi.replace('https://', '').replace('doi:', '').replace('doi.org/', '').replace('DOI:', '').lstrip(" ")
        if url and doi:
            links += 'ELECTR: {}; DOI: {}'.format(url, doi)
            dois += doi
        elif url:
            links += 'ELECTR: {}'.format(url)
        elif doi:
            links += 'DOI: {}'.format(doi)
            dois += doi
    
    links += '; ELECTR: https://ntrs.nasa.gov/citations/{}'.format(docID)  # STI LINK
        
    if 'meetings' in df.columns:
        meets = df['meetings']
        if isinstance(meets, list):         # Concat meeting info
            for entry in meets:
                if isinstance(entry, dict) and 'name' in entry and entry.get('name') != '':
                    if 'location' in entry and entry.get('location') != '':
                        meet = entry['name']
                        meet_loc = entry['location']
                        j += "; {}, {}".format(meet, meet_loc)
                    else:
                        j += "; {}.".format(meet)
    
    if isinstance(idents, list):       # Concat additional links from sourceIdentifiers field
        for entry in idents:
            if isinstance(entry, dict):
                if entry.get('type') == 'URL' and 'number' in entry:
                    if 'arXiv' in entry['number'] or 'arxiv.org' in entry['number']:
                        arxiv = entry['number'].replace('arXiv:','')
                        links += '; ARXIV: {}'.format(arxiv)
                    else:
                        url = entry['number']
                        links += '; ELECTR: {}'.format(url)
                if entry.get('type') == 'DOI' and 'number' in entry:
                    doi = entry['number'].replace('doi:', '')
                    links += '; DOI: {}'.format(doi)
                    dois += doi
                    
    if isinstance(dwnlds, list):       # FULL TEXT LINKS
        for entry in dwnlds:
            if isinstance(entry, dict) and 'links' in entry:
                if 'pdf' in entry['links']:
                    dwnld = entry['links']['pdf']
                    pdf = 'https://ntrs.nasa.gov{}'.format(dwnld)
                    links += '; PDF: {}'.format(pdf)
                if 'fulltext' in entry['links']:
                    dwnld = entry['links']['fulltext']
                    fulltext = 'https://ntrs.nasa.gov{}'.format(dwnld)
                    links += '; RAW: {}'.format(fulltext)
            
    if isinstance(funds, list):
        for entry in funds:
            if isinstance(entry, dict) and 'type' in entry and 'number' in entry:
                if entry['type'] == 'CONTRACT_GRANT':
                    fundNum = entry['number']
                    links += '; GRANT: {}'.format(fundNum)

    # Append metadata to lists for refstrings and curation/ingest
    pubs_ls.append(pubnames)
    vols_ls.append(volumes)
    dois_ls.append(dois)
    lsD.append(pubdate)
    lsJ.append(j.lstrip('; '))
    lsI.append(links.lstrip('; '))


# # Record creation and resolver service

In [ ]:
# Read the Excel file of records
dt = pd.read_excel(filepath + filename)

# Create a set of normalized identifiers from the Excel file; with whitespace stripped and converted to lowercase
identifiers_ls = [str(ident).strip() for ident in dt['ident']]

# Prepare a set of existing identifiers for quick lookup
existing_identifiers = set(identifiers_ls)

In [ ]:
ref_list = []  # reference strings to query ADS Reference Resolver Service
records = []   # records meetadata

# Iterate through the data and generate refstrings and create records for new records
for ident, A, D, T, pub, vol, doi, F, J, B, I, W, K, Q, X, source in zip(STI_ids, lsA, lsD, lsT, pubs_ls, vols_ls, dois_ls, lsF, lsJ, lsB, lsI, lsW, lsK, lsQ, lsX, sources):
    normalized_ident = str(ident).strip()  # Normalize the identifier

    # Check if the identifier is not in the existing set (new record)
    if normalized_ident not in existing_identifiers:
    
        if isinstance(A, list):  # Grab just the first 10 authors
            auth = '; '.join(A[:10])
        
        if D is not None:
            D = D[:4]  # Year = first 4 digits of pubdate

        if all(item is not None for item in [auth, D, T, doi]):   # author, year, title, and doi
            ref = {
                "refstr": "%s, %s, %s, doi: %s" % (auth, D, T, doi),
                "authors": auth,
                "year": D,
                "title": T,
                "doi": doi
            }
        elif all(item is not None for item in [auth, D, T, pub, vol]):  # author, year, title, journal, and volume
            ref = {
                "refstr": "%s, %s, %s, %s %s" % (auth, D, T, pub, vol),
                "authors": auth,
                "year": D,
                "title": T,
                "journal": "%s %s" % (pub, vol)
            }
        elif all(item is not None for item in [auth, D, T, pub]):  # author, year, title, and journal
            ref = {
                "refstr": "%s, %s, %s, %s" % (auth, D, T, pub),
                "authors": auth,
                "year": D,
                "title": T,
                "journal": pub
            }
        elif all(item is not None for item in [auth, D, T]):  # author, year, and title
            ref = {
                "refstr": "%s, %s, %s" % (auth, D, T),
                "authors": auth,
                "year": D,
                "title": T
            }
        elif all(item is not None for item in [D, T]):  # year and title
            ref = {
                "refstr": "%s, %s" % (D, T),
                "year": D,
                "title": T
            }
        elif all(item is not None for item in [auth, T, doi]):   # author, title, and doi
            ref = {
                "refstr": "%s, %s, doi: %s" % (auth, T, doi),
                "authors": auth,
                "title": T,
                "doi": doi
            }
        elif all(item is not None for item in [auth, T, pub, vol]):  # author, title, journal, and volume
            ref = {
                "refstr": "%s, %s, %s %s" % (auth, T, pub, vol),
                "authors": auth,
                "title": T,
                "journal": "%s %s" % (pub, vol)
            }
        elif all(item is not None for item in [auth, T, pub]):  # author, title, and journal
            ref = {
                "refstr": "%s, %s, %s" % (auth, T, pub),
                "authors": auth,
                "title": T,
                "journal": pub
            }   
        elif all(item is not None for item in [auth, T]):  # author and title
            ref = {
                "refstr": "%s, %s" % (auth, T),
                "authors": auth,
                "title": T
            }
        else:
            ref = {"refstr":""}

        ref_string = json.dumps(ref, ensure_ascii=False)
        ref_list.append(ref_string)

        record = {
            "ident": normalized_ident,
            "authors": '; '.join([a for a in A if a is not None]),
            "affiliations": '; '.join([f for f in F if f is not None]),
            "title": T,
            "pubdate": D,
            "publication": J,
            "abstract": B,
            "properties": I,
            "collection": W,
            "keywords": K,
            "subjectcategory": Q,
            "comment": X,
            "source": source,
            "refstring": ref_string
        }
        records.append(record)

In [ ]:
# -- Reference Resolver Service Setup

# ADS Prod API Token
token = 'pHazHxvHjPVPAcotvj7DIijROZXUjG5vXa2OaCQO'
domain = 'https://api.adsabs.harvard.edu/v1/'

# Reference Service API request, querying my 'references' list
def resolve(references):
    payload = {'parsed_reference': references}
    response = requests.post(
        url = domain + 'reference/xml',
        headers = {'Authorization': 'Bearer ' + token,
                 'Content-Type': 'application/json',
                 'Accept':'application/json'},
        data = json.dumps(payload))
    if response.status_code == 200:
        return json.loads(response.content)['resolved'], 200
    else:
        print('From reference status_code is ', response.status_code)
    return None, response.status_code

# -- Run Reference Resolver Service
references = ref_list
references = [ref.replace("\n"," ") for ref in references]
references = [json.loads(ref) for ref in references]

# Resolve my references, results in 'total results' list
total_results = []
print('Querying %d references with the Reference Service ...'%len(references))
for i in range(0, len(references), 16):
    results, status = resolve(references[i:i+16])
    if status == 200:
        if results:
            total_results += results
#     else:
#         break

## Output Results/Update Spreadsheet

In [ ]:
# Merge the records with the reference results and output the data
merged_data = []

if len(records)>0 and len(records)==len(total_results):
    for record, result in zip(records, total_results):
        merged_entry = {
            **record,  # Include all fields from the 'record' dictionary
            'refstring': result['refstring'],  # Include 'refstring' from the 'result' dictionary
            'score': result['score'],  # Include 'score' from the 'result' dictionary
            'bibcode': result['bibcode'],  # Include 'bibcode' from the 'result' dictionary
    #             **result,
        }
        merged_data.append(merged_entry)
    print('merged results')

    # Read the Excel file into a DataFrame
    existing_data = pd.read_excel(filepath + filename)

    # Create a DataFrame from the merged data, taking only records w/o pubdate if for no_pubs
    merged_data_df = pd.DataFrame(merged_data)
    if name == "no_pub":
        merged_data_df = merged_data_df[(pd.isnull(merged_data_df['pubdate']))]

    # Count non matched items in the merged data
    not_matched_count = (merged_data_df['bibcode'] == '...................').sum()
    matched_count = len(merged_data_df) - not_matched_count

    # Concatenate the existing data and merged data vertically
    combined_data = pd.concat([existing_data, merged_data_df], ignore_index=True)

    # Write the combined data back to the Excel file
    combined_data.to_excel(filepath + filename, index=False)

    print(f'Saved data to {filename}: \n\
          {len(merged_data)} new records \n\
            > {matched_count} new matched \n\
            > {not_matched_count} new not matched')
elif len(records)==0:
    print('No records to add')
else:
    print(f'records: {len(records)}')
    print(f'results: {len(total_results)}')

    # Output the data/results from a single harvest to xlsx
    df1 = pd.DataFrame(records)
    df2 = pd.DataFrame(total_results)

    outfile = name + "_review.xlsx"
    with pd.ExcelWriter(outfile) as writer:
        df1.to_excel(writer, sheet_name='records', index=False)
        df2.to_excel(writer, sheet_name='results', index=False)
    print(f'Saved data to {outfile}')

In [ ]:
## Add identifiers to 'all_data' set
all_data = "STI_all_data.xlsx"
# filename = "null_pub.xlsx"

# Load the first Excel sheet
sheet1 = pd.read_excel(filepath + filename)

# Load the second Excel sheet
sheet2 = pd.read_excel(filepath + all_data)

# Convert idents from each sheet to a list
idents_to_append = sheet1['ident'].tolist()
all_list = sheet2['ident'].tolist()

# Convert the all_list to a set to keep only unique values
all_set = set(all_list)

# Add only unique idents to the all_set
for ident in idents_to_append:
    if ident not in all_set:
        all_list.append(ident)
        all_set.add(ident)

# Write the new idents list back to all_data
df = pd.DataFrame({'ident': all_list})
df.to_excel(filepath + all_data, index=False)
print(f"Number of all records: {len(df)}")

## Stats
# Count the number of rows with 'STI' and 'CHORUS' in the 'source' column
sti_count = len(sheet1[sheet1['source'] == 'STI'])
chorus_count = len(sheet1[sheet1['source'] == 'CHORUS'])

# Count the number of rows with '...................' in the 'bibcode' column
null_bibcode_count = len(sheet1[sheet1['bibcode'] == '...................'])

# Count the number of rows with a different value (not null) in the 'bibcode' column
bibcode_count = len(sheet1[(sheet1['bibcode'].notnull()) & (sheet1['bibcode'] != '...................')])

# Print the counts
print(f'Stats for {name}')
print(f'Number of rows with "STI": {sti_count}')
print(f'Number of rows with "CHORUS": {chorus_count}')
print(f'Number of rows with no bibcode: {null_bibcode_count}')
print(f'Number of rows with a bibcode: {bibcode_count}')

## Additional Resolver 
Take this additional step to see if any previously unmatched items can be matched

In [ ]:
# Read the Excel file into a DataFrame
# filename = "2023_data.xlsx"
master_data = pd.read_excel(filepath + filename)

# Get rows with a bibcode value
rows_with_bibcode = master_data[master_data['bibcode'] != '...................']
count_with_bibcode = len(rows_with_bibcode)
print(f"Rows with a bibcode: {count_with_bibcode}")

# Get rows with no bibcode
rows_with_no_bibcode = master_data[master_data['bibcode'] == '...................']
count_no_bibcode = len(rows_with_no_bibcode)
print(f"Rows with no bibcode: {count_no_bibcode}")

In [ ]:
# Initialize an empty list for references
ref_list2 = []

# Iterate through rows with no bibcode
for index, row in rows_with_no_bibcode.iterrows():
    D = row["pubdate"]
    T = row["title"]
    A = row["authors"]
    properties_str = str(row["properties"])  # Convert to string
    doi_match = re.search(r'(10\.\d+\/\S+)', properties_str)
    if doi_match:
        doi = doi_match.group(0).rstrip(";")
    else:
        doi = None
    
    if isinstance(row["publication"], str):
        J = row["publication"].split(", ")
    else:
        J = []  
    
    if all(item is not None for item in [A, T, D, doi]) and not pd.isna(doi):
        ref = {
            "refstr": f"{A}, {T}, {str(D)}, {doi}",
            "authors": A,
            "title": T,
            "year": str(D),
            "doi": doi
        }
    elif all(item is not None for item in [A, T, D]):
        ref = {
            "refstr": f"{A}, {T}, {str(D)}",
            "authors": A,
            "title": T,
            "year": str(D)
        }
    elif doi is not None and not pd.isna(doi):
        ref = {
            "refstr": f"{doi}",
            "doi": doi
        }
    else:
        ref = {"refstr":""}
    
    ref_string = json.dumps(ref, ensure_ascii=False)
#     print(ref_string, '\n')
    ref_list2.append(ref_string)

# Reference Service API request, querying my 'references' list
# ADS Prod API Token
token = 'pHazHxvHjPVPAcotvj7DIijROZXUjG5vXa2OaCQO'
domain = 'https://api.adsabs.harvard.edu/v1/'
def resolve(references):
    payload = {'parsed_reference': references}
    response = requests.post(
        url = domain + 'reference/xml',
        headers = {'Authorization': 'Bearer ' + token,
                 'Content-Type': 'application/json',
                 'Accept':'application/json'},
        data = json.dumps(payload))
    if response.status_code == 200:
        return json.loads(response.content)['resolved'], 200
    else:
        print('From reference status_code is', response.status_code)
        return None, response.status_code

# Resolve my references, results in 'total results' list
references = [json.loads(ref) for ref in ref_list2]
total_results = []
print('Querying %d references with the Reference Service ...'%len(references))
for i in range(0, len(references), 16):
    results, status = resolve(references[i:i+16])
    if results:
        total_results += results
#     else:
#         break

# Save the results to excel
dt = pd.DataFrame(total_results)
dt.to_excel("ref_review.xlsx", index=False)
print("Saved ref results to ref_review.xlsx")